# Cat Breed Classification Using Deep Learning

## 1. Introduction

### 1.1. Definition of the Problem

This project aims to build and train a deep learning model to classify images of cats into their respective breeds. The task is framed as a multi-class classification problem, where each image is assigned a single breed label. Using principles from **Topic 5: Deep Learning for Computer Vision**, the model will leverage transfer learning and advanced techniques to achieve high accuracy.

The project also explores methods to optimise performance, including data augmentation, batch normalisation, and callbacks.

### 1.2. Dataset

- **`data/raw/cats.csv`**
  - A metadata file containing information about each image.
- **`data/raw/images/{breeds}`**
  - Contains subfolders named after each breed, with each subfolder holding the corresponding images of cats.

### 1.3. Workflow

The workflow follows **Chollet's Universal Workflow for Deep Learning**:
1. **Data Loading & Exploration:** Load and inspect the dataset.
2. **Data Preprocessing:** Split the dataset, apply augmentation, and preprocess images.
3. **Baseline Model:** Train a model using transfer learning with EfficientNet.
4. **Advanced Techniques:** Enhance the model with batch normalisation, callbacks, and the Functional API.
5. **Evaluation:** Test the model and visualise results with metrics and Grad-CAM.
6. **Conclusion:** Summarise results and propose improvements.

### 1.4. Advanced Techniques

The following advanced techniques from **Topic 7** are applied:
1. **Functional API:** To build a flexible and modular model.
2. **Batch Normalisation:** For stable and efficient training.
3. **Callbacks:** To monitor training with early stopping and learning rate scheduling.
4. **Grad-CAM:** To interpret predictions by visualising key image regions.

## 2. Data Loading & Exploration

### 2.1. Downloading the Dataset

First we must download the dataset from Kaggle, ensure that you have a `kaggle.json` file uploaded to your Google Drive (if on Google Colab), otherwise store it here in the root directory.

If the dataset is not already present in the environment, the notebook will:
1. Resolve the `kaggle.json`.
2. Configure the Kaggle API for authentication.
3. Download and extract the dataset into the `data/raw` directory.

This step ensures the dataset is available for exploration and preprocessing.

In [ ]:
import os
import pathlib
import sys
import shutil

# Define Dataset & API Key Paths
DATASET_DIR = pathlib.Path("data")
RAW_DIR = DATASET_DIR / "raw"
KAGGLE_JSON_LOCAL = "kaggle.json"
KAGGLE_JSON_DRIVE = "/content/drive/MyDrive/kaggle.json"

# Function to Check Dataset Presence
def is_dataset_present() -> bool:
    if RAW_DIR.exists() and any(RAW_DIR.iterdir()):
        print("Dataset Already Exists in `data/raw` - Skipping Download . . .")
        return True
    print("Dataset Not Found!")
    return False

# Function to Setup Kaggle Authentication
def setup_kaggle_auth() -> bool:
    if os.path.isfile(KAGGLE_JSON_LOCAL):
        print("Found `kaggle.json` Locally!")
        os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
        os.system(f"cp {KAGGLE_JSON_LOCAL} ~/.kaggle/")
        os.system("chmod 600 ~/.kaggle/kaggle.json")
        return True

    elif "google.colab" in sys.modules:
        print("`kaggle.json` Not Found Locally - Checking Google Drive . . .")
        from google.colab import drive
        drive.mount('/content/drive')

        if os.path.isfile(KAGGLE_JSON_DRIVE):
            print("Found `kaggle.json` in Google Drive!")
            os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
            os.system(f"cp {KAGGLE_JSON_DRIVE} ~/.kaggle/")
            os.system("chmod 600 ~/.kaggle/kaggle.json")
            return True
        else:
            print("`kaggle.json` Not Found in Google Drive! Please Add it to Your Root Drive.")
            return False

    else:
        print("`kaggle.json` Not Found Locally & Not in a Colab Environment!")
        print("Please Add `kaggle.json` to the Current Directory.")
        return False

# Function to Download & Extract Dataset
def download_and_extract_dataset() -> None:
    print("Downloading Dataset from Kaggle . . .")
    os.system("kaggle datasets download -d denispotapov/cat-breeds-dataset-cleared -p .")
    os.system("unzip -q cat-breeds-dataset-cleared.zip -d .")
    print("Dataset Downloaded & Extracted Successfully!")

# Function to Clean and Organize Dataset
def organise_dataset() -> None:
    # Rename 'dataset' to 'data'
    dataset_dir = pathlib.Path("dataset")
    if dataset_dir.exists() and dataset_dir.is_dir():
        dataset_dir.rename("data")

    # Move Files to 'data/raw'
    os.makedirs(RAW_DIR, exist_ok=True)
    for item in DATASET_DIR.iterdir():
        if item.is_file() or item.is_dir() and item.name != "raw":
            shutil.move(str(item), str(RAW_DIR))

    # Move Files from 'data/raw/data' to 'data/raw'
    nested_data_dir = RAW_DIR / "data"
    if nested_data_dir.exists() and nested_data_dir.is_dir():
        for item in nested_data_dir.iterdir():
            shutil.move(str(item), str(RAW_DIR))
        nested_data_dir.rmdir()  # Remove the now-empty 'data' folder

    # Delete ZIP File
    zip_path = pathlib.Path("cat-breeds-dataset-cleared.zip")
    if zip_path.exists():
        zip_path.unlink()

    print("Dataset Files Organized in `data/raw`!")

# Main Execution Flow
if not is_dataset_present():  # Check if Dataset is Already Present
    kaggle_auth = setup_kaggle_auth()  # Set-Up Kaggle Authentication
    if kaggle_auth:
        download_and_extract_dataset()
        organise_dataset()  # Organize Files in `data/raw`
    else:
        print("Kaggle Authentication Failed - Dataset Cannot Be Downloaded!")

### 2.2. Exploring the Dataset

To understand the dataset, we will:
1. Analyse the class distribution of breeds to check for imbalances.
2. Visualise a few sample images from different breeds.

In [ ]:
import pandas as pd

# Load Metadata
csv_path = RAW_DIR / "cats.csv"
cats_data = pd.read_csv(csv_path)

# Check for Missing Values
missing_values = cats_data.isnull().sum()
print("Missing Values in the Dataset:")
print(missing_values)

# Get Unique Breeds and Their Counts
breed_counts = cats_data['breed'].value_counts()
print("\nClass Distribution of Breeds:")
print(breed_counts)

# Get the Number of Unique Breeds
num_breeds = cats_data['breed'].nunique()
print(f"\nNumber of Unique Breeds: {num_breeds}")

### 2.3. Visualising the Dataset

We will display:
1. A bar plot of the breed distribution to identify dominant or underrepresented classes.
2. Randomly selected images from the dataset along with their breed labels.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import seaborn as sns

# Plot Breed Distribution with Adjusted Margins and Readable Labels
plt.figure(figsize=(12, 10))
sns.barplot(x=breed_counts.values, y=breed_counts.index, color="blue")  # Single color avoids warning
plt.title('Class Distribution of Cat Breeds', fontsize=16)
plt.xlabel('Count', fontsize=12)
plt.ylabel('Breed', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()  # Ensures the layout adjusts to avoid label overlap
plt.show()

# Visualise 6 Sample Images in a Grid
IMAGE_DIR = RAW_DIR / "images"

# Select the Most Common Breed
sample_breed = breed_counts.index[0]
all_images = os.listdir(os.path.join(IMAGE_DIR, sample_breed))

# Randomly Select 6 Images
random_images = random.sample(all_images, min(len(all_images), 6))  # Ensure no more than available images

# Set Number of Rows and Columns
cols = 3
rows = 2

# Adjust the Figure Size for Compact Display
plt.figure(figsize=(4 * cols, 4 * rows))

for i, img_file in enumerate(random_images):
    img_path = os.path.join(IMAGE_DIR, sample_breed, img_file)
    img = mpimg.imread(img_path)
    plt.subplot(rows, cols, i + 1)
    plt.imshow(img)
    plt.title(f"{sample_breed} {i+1}", fontsize=10)
    plt.axis('off')

plt.tight_layout()
plt.show()

## 3. Data Pre-Processing


### 3.1. Cleaning the Dataset
We will inspect the dataset for any missing or inconsistent values and handle them appropriately.

In [ ]:
# Inspect Missing Values
print("Missing Values Per Column:")
print(cats_data.isnull().sum())

# Drop Rows with Missing Values
cats_data = cats_data.dropna()
print(f"Dataset Size After Dropping Missing Values: {len(cats_data)}")

### 3.2. Splitting the Dataset

We dynamically split the dataset into `train`, `validation`, and `test` directories based on the existing subdirectories in `data/raw/images`. Each breed corresponds to a subdirectory, and the images are distributed into the three sets while maintaining class segregation.

In [ ]:
# Set to Use a Fraction of Data
USE_DATA_FRACTION = True

# Define Paths
SPLITS_DIR = DATASET_DIR / "splits"
SPLITS_DIR.mkdir(parents=True, exist_ok=True)

# Check if Directories Already Exist
if all((SPLITS_DIR / subset).exists() and any((SPLITS_DIR / subset).iterdir()) for subset in ['train', 'validation', 'test']):
    print("Dataset Already Organised Into Train, Validation & Test Folders. Skipping Re-Organisation!")
else:
    # Organise Dataset If Directories Are Missing or Empty
    print("Organising Dataset Into Train, Validation & Test Folders . . .")

    # Iterate Over Breeds (Folders in IMAGE_DIR)
    for breed_dir in IMAGE_DIR.iterdir():
        if breed_dir.is_dir():  # Ensure It’s a Directory
            breed = breed_dir.name
            images = list(breed_dir.glob("*.jpg"))

            # Shuffle Images for Random Distribution
            random.shuffle(images)

            # Use Fraction of Data if Enabled
            if USE_DATA_FRACTION:
                images = images[:int(len(images) * 0.2)] # Use 20% of Data

            # Split Data
            train_split = int(len(images) * 0.64)
            val_split = int(len(images) * 0.16)

            train_images = images[:train_split]
            val_images = images[train_split:train_split + val_split]
            test_images = images[train_split + val_split:]

            # Helper Function to Move Images
            def move_images(image_list, subset):
                subset_breed_dir = SPLITS_DIR / subset / breed
                subset_breed_dir.mkdir(parents=True, exist_ok=True)
                for img in image_list:
                    shutil.copy(img, subset_breed_dir / img.name)

            # Move Images to Appropriate Folders
            move_images(train_images, "train")
            move_images(val_images, "validation")
            move_images(test_images, "test")

    print("Dataset Re-Organised Successfully!")
    print(f"Train Images: {len(list((SPLITS_DIR / 'train').rglob('*.jpg')))}")
    print(f"Validation Images: {len(list((SPLITS_DIR / 'validation').rglob('*.jpg')))}")
    print(f"Test Images: {len(list((SPLITS_DIR / 'test').rglob('*.jpg')))}")

### 3.3. Transferring to Generators

Data generators streamline image preprocessing for training, validation, and testing. They:
1. **Rescale Pixel Values**: All pixel values are normalised to the `[0, 1]` range.
2. **Apply Augmentation**: Augmentation is applied to the training dataset to enhance model generalisation. Transformations include:
   - Random rotations
   - Width and height shifts
   - Zoom and shear
   - Horizontal flipping
3. **Load Data Dynamically**: Images and labels are dynamically loaded based on the directory structure in `SPLITS_DIR`. Labels are automatically assigned based on subdirectory names.

The resulting generators handle data preparation seamlessly for model training and evaluation.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define Data Generators
print("Setting Up Data Generators . . .")

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create Generators
train_generator = train_datagen.flow_from_directory(
    SPLITS_DIR / 'train',
    target_size=(128, 128),
    batch_size=64   ,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    SPLITS_DIR / 'validation',
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    SPLITS_DIR / 'test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Preserve Order for Evaluation
)

# Display Class Indices
print("Class Indices Mapping (Label to Class):")
print(train_generator.class_indices)

## 4. Model Architecture

### 4.1. Configuring the Model

In this step, we configure the deep learning model for cat breed classification. The process involves:

1. **Defining the Architecture**:
   - A pre-trained base model (e.g., EfficientNet or ResNet) is used as a feature extractor, leveraging knowledge from the ImageNet dataset.
   - Custom layers are added for the 67-class classification task.
   - Dropout is used for regularisation to prevent overfitting.
   - The final output layer uses softmax activation to predict probabilities for all 67 classes.

2. **Compiling the Model**:
   - **Loss Function**: Categorical cross-entropy for multi-class classification.
   - **Optimiser**: Adam for efficient optimisation.
   - **Metrics**: Accuracy as the evaluation metric.

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from pathlib import Path

# Dynamically Calculate Number of Classes
train_dir = SPLITS_DIR / 'train'
num_classes = len(list(train_dir.iterdir()))  # Count Subdirectories in 'train'

# Load Pre-Trained Base Model
print("Loading Pre-Trained ResNet50 Model . . .")
base_model = ResNet50(
    weights='imagenet',  # Pre-Trained On ImageNet
    include_top=False,   # Exclude The Original Classification Head
    input_shape=(128, 128, 3)  # Input Size For ResNet50
)

# Freeze The Base Model's Layers
print("Freezing Base Model Layers . . .")
base_model.trainable = False

# Build The Model Architecture
print("Building The Model Architecture . . .")
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Convert Feature Maps To A Single Vector
    Dropout(0.5),              # Add Dropout For Regularisation
    Dense(256, activation='relu'),  # Fully Connected Layer
    Dropout(0.3),                   # Add Another Dropout Layer
    Dense(num_classes, activation='softmax')  # Output Layer For Detected Classes
])

# Compile The Model
print("Compiling The Model . . .")
model.compile(
    optimizer='adam',  # Adam Optimiser
    loss='categorical_crossentropy',  # Multi-Class Classification Loss
    metrics=['accuracy']  # Evaluate Model Using Accuracy
)

# Display Model Summary
print("Displaying Model Summary . . .")
model.summary()

### 4.2. Training the Model

The model is trained using the training and validation datasets. This step includes:
1. **Data Generators**: Feeding data dynamically using `train_generator` and `validation_generator`.
2. **Callbacks**:
   - **Early Stopping**: Stops training if validation performance stops improving to prevent overfitting.
   - **Learning Rate Reduction**: Reduces the learning rate when the validation accuracy plateaus.
3. **Training Configuration**:
   - Number of epochs and batch size are defined for efficient training.
   - Validation data is evaluated after each epoch to monitor performance.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define Callbacks
print("Setting Up Callbacks . . .")
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor Validation Loss
    patience=8,          # Increase Patience for Deeper Models
    restore_best_weights=True  # Restore Weights From Best Epoch
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Monitor Validation Loss
    factor=0.2,          # Reduce Learning Rate By Factor Of 0.2
    patience=5,          # Trigger LR Reduction After 5 Epochs Without Improvement
    min_lr=1e-6          # Set Minimum Learning Rate
)

# Adjust Training Parameters
print("Starting Model Training . . .")
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr],
    verbose=1,
)

# Training Completed
print("Model Training Completed.")